In [1]:
import numpy as np
import scipy.signal
import matplotlib.pyplot as plt
import scipy.signal
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.optimize import least_squares
from scipy.integrate import simps
from math import factorial
from numpy.polynomial.polynomial import Polynomial

t = np.linspace(0, 0.004*35000, 35001)
delta_t = 0.004
n_values = np.arange(11)
gauss_params = [5000, 0.25, 8e+00, 100, 0.2, 9, 50, 0.65, 10, 1.5e+00]
model_params = [0.008, 11]


#Define simulated data
def skewed_IRF_with_small_peak(t, A1, sigma1, mu1, A2, sigma2, mu2, A3, sigma3, mu3, b):
    # Main Gaussian peak
    main_peak = A1 / (sigma1 * np.sqrt(2 * np.pi)) * np.exp(-((t - mu1)**2) / (2 * sigma1**2))
    
    # Smaller Gaussian peak to the right-hand side
    small_peak = A2 / (sigma2 * np.sqrt(2 * np.pi)) * np.exp(-((t - mu2)**2) / (2 * sigma2**2))

    smaller_peak =  A3 / (sigma3 * np.sqrt(2 * np.pi)) * np.exp(-((t - mu3)**2) / (2 * sigma3**2))
    
    # Combine the main peak, the small peak, and the baseline offset
    irf = main_peak + small_peak +smaller_peak + b

    return irf

def skewed_IRF_with_small_peak_noise (t, A1, sigma1, mu1, A2, sigma2, mu2, A3, sigma3, mu3, b):
    # Main Gaussian peak
    main_peak = A1 / (sigma1 * np.sqrt(2 * np.pi)) * np.exp(-((t - mu1)**2) / (2 * sigma1**2))
    
    # Smaller Gaussian peak to the right-hand side
    small_peak = A2 / (sigma2 * np.sqrt(2 * np.pi)) * np.exp(-((t - mu2)**2) / (2 * sigma2**2))

    smaller_peak =  A3 / (sigma3 * np.sqrt(2 * np.pi)) * np.exp(-((t - mu3)**2) / (2 * sigma3**2))
    
    # Combine the main peak, the small peak, and the baseline offset
    irf = main_peak + small_peak +smaller_peak + b
    noise_level = 0.00005 * max(irf)
    noise = np.random.normal(loc=0, scale=noise_level, size=irf.shape)
    return np.clip(irf + noise, a_min=0, a_max=None)

#defined an IRF with noise as this is what you would experimentally measure as the IRF, will be used later

def multi_exp_model_truncated(t, C, tau):
    # Generate the model based on two exponential decays
    model = C * np.exp(-t / tau) 
    # Perform convolution of the model with the input response function (irf)
    raw_convolution = scipy.signal.fftconvolve(model, IRF_no_noise, mode='full')[:len(t)]
    # Calculate noise level as a percentage of the maximum point of the raw convolution
    noise_level = 0.0001 * max(raw_convolution)
    # Generate random noise
    noise = np.random.normal(loc=0, scale=noise_level, size=raw_convolution.shape)
    # Add noise to the raw convolution
    noisy_convolution = raw_convolution + noise
    max_idx = np.argmax(noisy_convolution)
    t_truncated = t[max_idx:] - t[max_idx]  # Adjust time to start from zero at the max index
    convoluted_truncated = noisy_convolution[max_idx:]
    IRF_truncated = IRF_noisy[max_idx:]
    return t_truncated, convoluted_truncated, IRF_truncated



IRF_noisy = skewed_IRF_with_small_peak_noise(t, *gauss_params)
IRF_no_noise = skewed_IRF_with_small_peak(t, *gauss_params)

t_adjusted, simulated_data, IRF_adjusted = multi_exp_model_truncated(t, *model_params)


In [2]:

# Definitions of your functions (unchanged)
def single_exponential_approx(t, A, tau, c):
    single_exp_decay_1 = A * np.exp(- (t / tau)) - c
    raw_convolution = scipy.signal.fftconvolve(single_exp_decay_1, IRF_noisy, mode='full')[:len(t)]
    return raw_convolution

def single_exp_model_truncated(t, C, tau, c):
    model = C * np.exp(-t / tau) - c
    # Perform convolution of the model with the input response function (irf)
    raw_convolution = scipy.signal.fftconvolve(model, IRF_noisy, mode='full')[:len(t)]
    # Calculate noise level as a percentage of the maximum point of the raw convolution
    noise_level = 0.0001 * max(raw_convolution)
    # Generate random noise
    noise = np.random.normal(loc=0, scale=noise_level, size=raw_convolution.shape)
    # Add noise to the raw convolution
    noisy_convolution = raw_convolution + noise
    max_idx = np.argmax(noisy_convolution)
    t_truncated = t[max_idx:] - t[max_idx]  # Adjust time to start from zero at the max index
    convoluted_truncated = noisy_convolution[max_idx:]
    IRF_truncated = IRF_noisy[max_idx:]
    return t_truncated, convoluted_truncated, IRF_truncated

def Pure_single_exp(t, A, tau, c):
    return A * np.exp(- (t / tau)) - c

def binomial_coefficient(n, k):
    # Calculate the binomial coefficient "n choose k"
    return factorial(n) // (factorial(k) * factorial(n - k))

def integrate_for_n(t, data, n):
    # Original provided function to calculate the integral for the nth power
    #background = 6.5
    #background = np.mean(-data[int(0.0001*len(data)):])
    #corrected_data = data - noise
         # Calculate the background as mean of the last 10% of the data points
    #n_background = round(len(data) * 0.4)
    #background = np.mean(data[-n_background:])
    fit_region = data[-int(0.451*len(data)):]
    model = Polynomial.fit(t[-int(0.451*len(t)):], fit_region, deg=2)
    background_fit = model(t)
    corrected_data = data - background_fit 
    integrand_num = (t**n) * corrected_data
    integrand_dem = corrected_data
    integral_num = simps(integrand_num, t)
    integral_dem = simps(integrand_dem, t)
    return integral_num  


def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())



mu_k_values = [integrate_for_n(t_adjusted, simulated_data, n) for n in n_values]

print(mu_k_values)


[104043.78640510264, 1091799.6338434655, 21896701.56436999, 621851452.916474, 21958765630.11307, 889084443967.8032, 37998690696527.0, 1427380096233197.0, 3742007942403491.0, -1.0468471146870245e+19, -2.14490957006832e+21]


In [3]:
def multi_exp_model_fitting(t, C, tau):
    # Generate the model based on two exponential decays
    model = C * np.exp(-t / tau)
    # Perform convolution of the model with the input response function (irf)
    raw_convolution = scipy.signal.fftconvolve(model, IRF_no_noise, mode='full')[:len(t)]
    # Calculate noise level as a percentage of the maximum point of the raw convolution
    noise_level = 0.0001 * max(raw_convolution)
    # Generate random noise
    noise = np.random.normal(loc=0, scale=noise_level, size=raw_convolution.shape)
    # Add noise to the raw convolution
    noisy_convolution = raw_convolution + noise
    return noisy_convolution


fitting_data =  multi_exp_model_fitting(t, *model_params)
fitting_data_new = np.abs(fitting_data)
print(fitting_data)
print(fitting_data_new)

[-1.83314671  0.81431556 -0.51405186 ... 32.58653002 32.35278928
 33.59655344]
[ 1.83314671  0.81431556  0.51405186 ... 32.58653002 32.35278928
 33.59655344]


In [4]:
# Initial fitting to get starting parameters
Trial_single = [1, 1, 1e-3]

popt, pcov = curve_fit(single_exponential_approx, t, fitting_data_new, p0=Trial_single)

a_opt, tau_opt, c_opt = popt

# Iterative process starts here
max_iterations = 100
tolerance = 1e-7
previous_rmse = 6e30
current_params = a_opt, tau_opt

for iteration in range(max_iterations):
    # Use current parameters to calculate F_single
    T_large = np.linspace(0.004*35000, 0.004*1000000, 965001)
    F_single = Pure_single_exp(T_large, *current_params, c_opt)

    # Calculate moments and new parameters
    mu_0_calc = integrate_for_n(t_adjusted, simulated_data, 0) + integrate_for_n(T_large, F_single, 0)
    m_0_calc = integrate_for_n(t_adjusted, IRF_adjusted, 0)
    mu_1_calc = integrate_for_n(t_adjusted, simulated_data, 1) + integrate_for_n(T_large, F_single, 1)
    m_1_calc = integrate_for_n(t_adjusted, IRF_adjusted, 1)
    G1 = mu_0_calc / m_0_calc
    G2 = (1 / m_0_calc)*(mu_1_calc - (mu_0_calc / m_0_calc)*(m_1_calc))
    new_a = (G1**2) / G2
    new_tau = G2 / G1
    new_params = [new_a, new_tau]


    t_adjusted_2, single_Data, irf = single_exp_model_truncated(t, *new_params, c_opt)

    
    # Compute mu_est_values for RMSE calculation
    mu_est_values = [integrate_for_n(t_adjusted_2, single_Data, n) for n in n_values]
    current_rmse = rmse(np.array(mu_est_values), np.array(mu_k_values))
   
    # Check for convergence
    if abs(previous_rmse - current_rmse) < tolerance:
        break
    previous_rmse = current_rmse
    current_params = new_params  # Update parameters for next iteration

# Output the results
print(f"Optimized parameters: A={new_a}, tau={new_tau}")
print(f"Final RMSE: {current_rmse}")



Optimized parameters: A=58.83522296023517, tau=9.519306392855068
Final RMSE: 1.7837801531399843e+24


In [67]:

#fit is not getting any better therefore we add one component

#define new double exp model
def Pure_multi_exp(t, C, a1, tau1, a2, tau2, c):
    a2 = 1 - a1
    return C*(a1 * np.exp(- (t / tau1)) + a2 * np.exp(- (t / tau2))) - c
   
def multi_exp_model_truncated_it(t, C, a1, tau1, a2, tau2, c):
    # Generate the model based on two exponential decays
    model = C * (a1 * np.exp(-t / tau1) + a2 * np.exp(-t / tau2)) - c
    # Perform convolution of the model with the input response function (irf)
    raw_convolution = scipy.signal.fftconvolve(model, IRF_noisy, mode='full')[:len(t)]
    # Calculate noise level as a percentage of the maximum point of the raw convolution
    noise_level = 0.0001 * max(raw_convolution)
    # Generate random noise
    noise = np.random.normal(loc=0, scale=noise_level, size=raw_convolution.shape)
    # Add noise to the raw convolution
    noisy_convolution = raw_convolution + noise
    max_idx = np.argmax(noisy_convolution)
    t_truncated = t[max_idx:] - t[max_idx]  # Adjust time to start from zero at the max index
    convoluted_truncated = noisy_convolution[max_idx:]
    IRF_truncated = IRF_noisy[max_idx:]
    return t_truncated, convoluted_truncated, IRF_truncated

def multi_exp_model_it(t, C, a1, tau1, tau2, c):
    a2 = 1 - a1
    # Generate the model based on two exponential decays
    model = C * (a1 * np.exp(-t / tau1) + a2 * np.exp(-t / tau2)) - c
    # Perform convolution of the model with the input response function (irf)
    raw_convolution = scipy.signal.fftconvolve(model, IRF_noisy, mode='full')[:len(t)]
    # Calculate noise level as a percentage of the maximum point of the raw convolution
    noise_level = 0.0001 * max(raw_convolution)
    # Generate random noise
    noise = np.random.normal(loc=0, scale=noise_level, size=raw_convolution.shape)
    # Add noise to the raw convolution
    noisy_convolution = raw_convolution + noise
    return noisy_convolution


Trial_double = [1, 1, 1, 1, 1e-3]
bounds = ([0, 0, 0, 0, 0], [np.inf, 1, np.inf, np.inf, np.inf]) 
popt_2, pcov_2 = curve_fit(multi_exp_model_it, t, fitting_data_new, p0=Trial_double, bounds=bounds)
C_opt, a1_opt, tau1_opt, tau2_opt, c2_opt = popt_2

a2_opt = 1 - a1_opt

max_iterations_multi = 100
current_params_multi = popt_2
tolerance_multi = 1e-6
previous_rmse_multi = 6e30

def residuals(vars, G1, G2, G3, G4):
    a1, tau1, a2, tau2 = vars[0], vars[1], vars[2], vars[3]
    return [
        a1 * tau1 + a2 * tau2 - G1,
        a1 * tau1**2 + a2 * tau2**2 - G2,
        a1 * tau1**3 + a2 * tau2**3 - G3,
        a1 * tau1**4 + a2 * tau2**4 - G4
    ]

for iteration in range(max_iterations_multi):
    # Use current parameters to calculate F_single
    T_large = np.linspace(0.004*35000, 0.004*1000000, 965001)
    F_Multi = Pure_multi_exp(T_large, C_opt, a1_opt, tau1_opt, a2_opt, tau2_opt, c2_opt)

    # Calculate moments and new parameters
    mu_0_calc = integrate_for_n(t_adjusted, simulated_data, 0) + integrate_for_n(T_large, F_Multi, 0)
    m_0_calc = integrate_for_n(t_adjusted, IRF_adjusted, 0)
    mu_1_calc = integrate_for_n(t_adjusted, simulated_data, 1) + integrate_for_n(T_large, F_Multi, 1)
    m_1_calc = integrate_for_n(t_adjusted, IRF_adjusted, 1)
    mu_2_calc = integrate_for_n(t_adjusted, simulated_data, 2) + integrate_for_n(T_large, F_Multi, 2)
    m_2_calc = integrate_for_n(t_adjusted, IRF_adjusted, 2)
    mu_3_calc = integrate_for_n(t_adjusted, simulated_data, 3) + integrate_for_n(T_large, F_Multi, 3)
    m_3_calc = integrate_for_n(t_adjusted, IRF_adjusted, 3)

    G1 = mu_0_calc / m_0_calc
    G2 = (1 / m_0_calc)*(mu_1_calc - G1*m_1_calc)
    G3 = (1 / m_0_calc)*((mu_2_calc / 2) - (G1*m_2_calc / 2) - G2*m_1_calc)
    G4 = (1 / m_0_calc)*((mu_3_calc / 6) - (G1*m_3_calc / 6) - (G2*m_2_calc / 2) -G3*mu_1_calc)

    params = a1_opt, tau1_opt, a2_opt, tau2_opt


    
# Perform least squares optimization
    result = least_squares(residuals, params, args=(G1, G2, G3, G4))
    
# Display the results

    t_truncated_3, F_multi_approx, irf_3 = multi_exp_model_truncated_it(t, C_opt, result.x[0], result.x[1], result.x[2], result.x[3], c2_opt)
   
    mu_est_values_multi = [integrate_for_n(t_truncated_3, F_multi_approx, n) for n in n_values]
    current_rmse_multi = rmse(np.array(mu_est_values_multi), np.array(mu_k_values))

    # Check for convergence
    if abs(previous_rmse_multi - current_rmse_multi) < tolerance_multi:
        break
    previous_rmse_multi = current_rmse_multi
    a1_opt, tau1_opt, a2_opt, tau2_opt = result.x  # Update parameters for next iteration

# Output the results
print(result.x)
print(f"Final RMSE: {current_rmse_multi}")

LinAlgError: SVD did not converge in Linear Least Squares